Fixed length FIFO Queue for Redis

In [158]:
import redis
class redisFIFOQ:
    
    newsQueueSize = 100
    newsQueueName = "news"
    
    def __init__(self):
        self.db = redis.Redis()
    
    def put(self, key, value):
        if not isinstance(value,dict):
            print ("Value needs to be a dictionary " + key)
            return False
        if not isinstance(key,str):
            print ("Key needs to be a string " + key)
            return False
        self.db.hmset(key, value)
        return True
    
    def get(self, key):
        if not isinstance(key,str):
            print ("Key needs to be a string " + key)
            return False
        elif not self.db.exists(key):
            print ("Key doesnt exist " + key)
            return False
        
        val = self.db.hgetall(key)
        return val
    
    def delete(self, key):
        if not self.db.exists(key):
            print ("Key doesnt exist " + key)
            return False
        
        self.db.delete(key)
        return True
    
    
    
    def getLengthOfQueue(self):
        return self.db.llen(self.newsQueueName)
        
    def deleteAll(self):
        for key in self.db.keys():
            # delete the key
            self.db.delete(key)
            
    def checkIfExists(self, key):
        if self.db.exists(key):
            return True
        else:
            return False
        
    #pushed value to top of news queue
    def pushIntoQueue(self, value):
        
        self.db.lpush(self.newsQueueName,value)
        return True
    
    #pushed value from end of queue (Pops oldest news)
    def pop(self):
        val = self.db.rpop(self.newsQueueName)
        return val 
    
    #Get values from start to end positions of newws queue
    def getValuesFromQueue(self, start, end):
        val = self.db.lrange(self.newsQueueName, start, end)
        return val
    
    #Get 
    def getEntireNewsQueue(self):
        return self.getValuesFromQueue(0,self.newsQueueSize-1)

In [159]:
db = redisFIFOQ()

In [160]:
db.checkIfExists('Jay')
db.put("Jay", {"surname" : "jaylohokare"})
db.get("Jay")
db.delete("Jay")


True

In [166]:
db.pushIntoQueue("4")

True

In [168]:
db.pop()

b'3'

In [169]:
db.getEntireNewsQueue()

[b'4', b'4', b'4', b'4', b'1', b'2', b'3', b'4', b'4']